In [1]:
import pandas as pd
import os
import sys
import yaml
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
import mlflow
import pickle


In [2]:
df=pd.read_csv("/Users/pratik.kujur/Desktop/Projects/Mlops-end-to-end/data/raw/winequality.csv")
df.head()

,Unnamed: 0,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,is_red
0,0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,4,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5,1


### Data Pre-Processing

In [3]:
%pwd

'/Users/pratik.kujur/Desktop/Projects/Mlops-end-to-end/experiments'

In [4]:
os.chdir("../")
%pwd

'/Users/pratik.kujur/Desktop/Projects/Mlops-end-to-end'

In [5]:
params=yaml.safe_load(open("/Users/pratik.kujur/Desktop/Projects/Mlops-end-to-end/params.yaml"))['preprocess']

In [6]:
def DataPreprocessing(input_path,output_path):
    df=pd.read_csv(input_path)
    print("Raw Data \n",df.head())
    # add preprocessing step here

    unecessary_col=['Unnamed: 0']
    df.drop(labels=unecessary_col,axis='columns',inplace=True)
    
    os.makedirs(os.path.dirname(output_path),exist_ok=True)
    df.to_csv(output_path,index=True,header=True)
    print("Preprocessed Data \n",df.head())
    print(f"Preprocessed data is saved in this location -> {output_path}")

In [11]:
params_trainer=yaml.safe_load(open("/Users/pratik.kujur/Desktop/Projects/Mlops-end-to-end/params.yaml"))['train']

params_evalution=yaml.safe_load(open("/Users/pratik.kujur/Desktop/Projects/Mlops-end-to-end/params.yaml"))["evalution"]

params_track=yaml.safe_load(open("/Users/pratik.kujur/Desktop/Projects/Mlops-end-to-end/params.yaml"))["mlflow"]

### DataSplit

In [8]:
def DataSplit(params_trainer,params_evalution,processed_path,
              X_train_reg_path,y_train_reg_path,X_train_clf_path,y_train_clf_path,
              y_test_reg_path,y_test_clf_path,X_test_reg_path,X_test_clf_path):

    df=pd.read_csv(processed_path)

    y_reg=df['quality']
    y_clf=df['is_red']

    X_clf=df.iloc[:,1:12]
    X_reg=df.iloc[:,1:11]

    X_train_reg,X_test_reg,y_train_reg,y_test_reg=train_test_split(X_reg,y_reg,test_size=0.2,random_state=42)
    X_train_clf,X_test_clf,y_train_clf,y_test_clf=train_test_split(X_clf,y_clf,test_size=0.2,random_state=42)

    os.makedirs(os.path.dirname(X_train_reg_path),exist_ok=True)
    os.makedirs(os.path.dirname(X_test_reg_path),exist_ok=True)
    os.makedirs(os.path.dirname(y_train_reg_path),exist_ok=True)
    os.makedirs(os.path.dirname(y_test_reg_path),exist_ok=True)
    os.makedirs(os.path.dirname(X_train_clf_path),exist_ok=True)
    os.makedirs(os.path.dirname(X_test_clf_path),exist_ok=True)
    os.makedirs(os.path.dirname(y_train_clf_path),exist_ok=True)
    os.makedirs(os.path.dirname(y_test_clf_path),exist_ok=True)
    
    X_train_reg.to_csv(X_train_reg_path,index=False,header=None)
    X_test_reg.to_csv(X_test_reg_path,index=False,header=None)
    y_train_reg.to_csv(y_train_reg_path,index=False,header=None)
    y_test_reg.to_csv(y_test_reg_path,index=False,header=None)
    X_train_clf.to_csv(X_train_clf_path,index=False,header=None)
    X_test_clf.to_csv(X_test_clf_path,index=False,header=None)
    y_train_clf.to_csv(y_train_clf_path,index=False,header=None)
    y_test_clf.to_csv(y_test_clf_path,index=False,header=None)
    

    print(f"Data for Taining are saved at ->{params_trainer}")
    print(f"Data for evalution are saved at ->{params_evalution}")

In [9]:
def ModelTrainer(reg,clf,model_reg_path,model_clf_path,
                 X_train_reg,y_train_reg,X_train_clf,y_train_clf):
    X_train_clf=pd.read_csv(X_train_clf)
    y_train_clf=pd.read_csv(y_train_clf)
    X_train_reg=pd.read_csv(X_train_reg)
    y_train_reg=pd.read_csv(y_train_reg)
    
    reg.fit(X_train_reg,y_train_reg)
    os.makedirs(os.path.dirname(model_reg_path),exist_ok=True)
    pickle.dump(reg,open(model_reg_path,'wb'))

    clf.fit(X_train_clf,y_train_clf)
    os.makedirs(os.path.dirname(model_clf_path),exist_ok=True)
    pickle.dump(clf,open(model_clf_path,'wb'))

    print(f"Regression model is saved at -> {model_reg_path}",f"Classification model is saved at -> {model_clf_path}")


In [ ]:
def ModelEvalution():
    reg_params=reg.get_params()
    clf_params=clf.get_params()

    # Mlflow Tracking
    mlflow.set_registry_uri(uri=uri)
    mlflow.set_experiment(experiment_name="Multioutput_Model")
    with mlflow.start_run():
    mlflow.log_params(reg_params)
    mlflow.log_params(clf_params)

In [ ]:
if __name__=="__main__":
    DataPreprocessing(params['input'],params['output'])
    DataSplit(params_trainer,params_evalution,params['output'],
              params_trainer['X_train_reg'],params_trainer['y_train_reg'],params_trainer['X_train_clf'],params_trainer['y_train_clf'],
              params_evalution['y_test_reg'],params_evalution['y_test_clf'],params_evalution['X_test_reg'],params_evalution['X_test_clf'])
    reg=RandomForestRegressor()
    clf=RandomForestClassifier()
    ModelTrainer(reg,clf,params_trainer['model_reg'],params_trainer['model_clf'],
                params_trainer['X_train_reg'],params_trainer['y_train_reg'],params_trainer['X_train_clf'],params_trainer['y_train_clf'])

Raw Data 
    Unnamed: 0  fixed acidity  volatile acidity  citric acid  residual sugar  \
0           0            7.4              0.70         0.00             1.9   
1           1            7.8              0.88         0.00             2.6   
2           2            7.8              0.76         0.04             2.3   
3           3           11.2              0.28         0.56             1.9   
4           4            7.4              0.66         0.00             1.8   

   chlorides  free sulfur dioxide  total sulfur dioxide  density    pH  \
0      0.076                 11.0                  34.0   0.9978  3.51   
1      0.098                 25.0                  67.0   0.9968  3.20   
2      0.092                 15.0                  54.0   0.9970  3.26   
3      0.075                 17.0                  60.0   0.9980  3.16   
4      0.075                 13.0                  40.0   0.9978  3.51   

   sulphates  alcohol  quality  is_red  
0       0.56      9.4       

/Users/pratik.kujur/Desktop/Projects/Mlops-end-to-end/.venv/lib/python3.10/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/pratik.kujur/Desktop/Projects/Mlops-end-to-end/.venv/lib/python3.10/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Regression model is saved at -> models/model_1_reg.pkl Classification model is saved at -> models/model_1_clf.pkl
